In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, models
import torch.optim as optim
import torch.nn.functional as F
import os
import pandas as pd
from torch.utils.data import Dataset, random_split, DataLoader
from skimage import io
from sklearn.preprocessing import LabelEncoder
from PIL import Image

import gc

gc.collect()

torch.cuda.empty_cache()

# 0) Initializing All The Necessary Dir Paths

In [2]:
dataset_path_dir = 'Dog-Breed-Dataset/'
train_path_dir = os.path.join(dataset_path_dir, 'train')
test_path_dir = os.path.join(dataset_path_dir, 'test')
labels_file_path = os.path.join(dataset_path_dir, 'labels.csv')

In [3]:
labels_file_path

'Dog-Breed-Dataset/labels.csv'

In [4]:
df = pd.read_csv(labels_file_path)
df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [5]:
df['breed'].nunique()

120

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


/home/dev_ashish/anaconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


## HyperParameter

In [7]:
BATCH_SIZE = 4
LEARNING_RATE = 0.001
EPOCHS = 10
IMG_SIZE = 120
IN_CHANNELS = 3
NUM_CLASSES = 120

# 1) Preparing Our Custom Dataset

In [8]:
class DogsBreedDataset(Dataset):
    def __init__(self, csv_file, train_dir, transform = None):
        self.annotations = pd.read_csv(csv_file)
        self.train_dir = train_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.annotations)
    
    
    def __getitem__(self, index):
        # index = rows, 0 is first column that is image name
        # index we dont choose, pytorch do it for us
        img_path = os.path.join(self.train_dir, self.annotations.iloc[index, 0])
        image = Image.open(f"{img_path}.jpg")
        
        #image = io.imread(f"{img_path}.jpg")
        
        # index = rows, 1 is second column that is labels name
        # index we dont choose, pytorch do it for us
        
        lr = LabelEncoder()
        transformed_labels = lr.fit_transform(self.annotations['breed'])
        dataset.annotations['breed'] = transformed_labels
        
        labels = torch.tensor(self.annotations.iloc[index, 1])
        
        if self.transform:
            image = self.transform(image)
            
        return (image, labels)

# 2) Preparing Transformations For Images

In [9]:
transformations = transforms.Compose([
    transforms.RandomResizedCrop(size = IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.465,0.406],
                        [0.229, 0.224, 0.225])
])

# 2) Load the Dataset

In [10]:
dataset = DogsBreedDataset(csv_file = labels_file_path, train_dir = train_path_dir, 
                           transform = transformations)

In [11]:
dataset.__len__()

10222

In [12]:
dataset

# 3) Split the Dataset

In [13]:
test_len = dataset.__len__() - 7000

In [14]:
train_ds, test_ds = random_split(dataset = dataset, lengths = [7000, test_len])

In [15]:
print(train_ds)
print(test_ds)
print(len(train_ds))
print(len(test_ds))

7000
3222


# 4) Data Loader

Now we have already prepared our custom dataset and splited it, now its time to load it in memory

In [16]:
train_loader = DataLoader(dataset = train_ds, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(dataset = test_ds, batch_size = BATCH_SIZE, shuffle = True)

# 5) Define Model

In [17]:
model = models.vgg16(pretrained = True)
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## 6) Freeze pre-trained model parameter to avoid backpropogation through them

In [18]:
for parameter in model.parameters():
    parameter.requires_grad = False

In [19]:
from collections import OrderedDict

# Build custom classifier
classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 5000)),
                                        ('relu', nn.ReLU()),
                                        ('drop', nn.Dropout(p = 0.5)),
                                        ('fc2', nn.Linear(5000, NUM_CLASSES)),
                                        ('output', nn.Softmax())]))

model.classifier = classifier

In [20]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# 7) Define Loss and Optimizer

In [21]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = model.classifier.parameters(), lr = LEARNING_RATE) # passing classifier.parameters()

# Train The Model

As Dataset is ready, splitted and loaded in Data Loader, Model is defined, Loss and Optimizer is Ready.
So Lets Train...

In [23]:
for epoch in range(EPOCHS):
    
    model.train()      # Training Mode Activated
    
    for images, labels in iter(train_loader):      # Iter "seems" to be optional
        images = images.to(device)
        labels = labels.to(device)
        
        # Reset the optimizer to zero
        optimizer.zero_grad()
        
        # send images to model for training
        preds = model(images)
        
        # Compute Loss
        loss = loss_func(preds, labels)
        
        # Compute Gradients
        loss.backward()
        
        # Update Parameters
        optimizer.step()
        

/home/dev_ashish/anaconda3/lib/python3.9/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [27]:
def check_accuracy(loader, model):
    if loader == train_loader:
        print("Checking accuracy on Training Data")
    else:
        print("Checking accuracy on Test Data")
        
    num_correct = 0
    num_samples = 0
    
    model.eval()    # go to evaluation mode
    
    with torch.no_grad():          # Turn off gradients calculation to save memory
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)
            scores = model(images)
            
            _, predictions = scores.max(1)
            num_correct += (predictions == labels).sum()
            num_samples += predictions.size(0)
            
        print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct) / float(num_samples) * 100:2f}")
        
    model.train()
    
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on Training Data


/home/dev_ashish/anaconda3/lib/python3.9/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Got 50 / 7000 with accuracy 0.714286
Checking accuracy on Test Data
Got 33 / 3222 with accuracy 1.024209
